# Library import

In [3]:
import cv2
import numpy as np
import os
import torch

# 삼각형 이미지 생성 및 저장 

In [16]:
def draw_random_triangle(image_size=(64, 64), line_thickness=2):
    image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    height, width = image.shape[:2]
    
    points = np.array([
        [np.random.randint(0, width//2), np.random.randint(0, height//2)],
        [np.random.randint(width//2, width), np.random.randint(height//2, height)],
        [np.random.randint(0, width//2), np.random.randint(height//2, height)], 
        [np.random.randint(width//2, width), np.random.randint(0, height//2)]
    ])
    
    selected_points = points[np.random.choice(points.shape[0], 3, replace=False)]
    
    cv2.polylines(image, [selected_points], isClosed=True, color=(255, 255, 255), thickness=line_thickness)
    return image

# 사각형 이미지 생성 및 저장 

In [19]:
def draw_random_rectangle(image_size=(64, 64), line_thickness=2):
    image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    height, width = image.shape[:2]
    
    pt1 = (np.random.randint(0, width), np.random.randint(0, height))
    pt2 = (np.random.randint(0, width), np.random.randint(0, height))
    
    cv2.rectangle(image, pt1, pt2, color=(255, 255, 255), thickness=line_thickness)
    return image

# 원형 이미지 생성 및 저장 

In [17]:
def draw_random_circle(image_size=(64, 64), min_radius=5, max_radius=20, line_thickness=2):
    image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    height, width = image.shape[:2]
    
    radius = np.random.randint(min_radius, max_radius)
    center_x = np.random.randint(radius, width - radius)
    center_y = np.random.randint(radius, height - radius)
    center = (center_x, center_y)
    
    
    cv2.circle(image, center, radius, color=(255, 255, 255), thickness=line_thickness)
    return image

# 디렉토리 생성 및 이미지 저장 

In [20]:
triangle_dir = '/home/juneyub12/Desktop/sample/triangle_images'
rectangle_dir = '/home/juneyub12/Desktop/sample/rectangle_images'
circle_dir = '/home/juneyub12/Desktop/sample/circle_images'

os.makedirs(triangle_dir, exist_ok=True)
os.makedirs(rectangle_dir, exist_ok=True)
os.makedirs(circle_dir, exist_ok=True)

# 이미지 생성 및 저장
for i in range(1, 1001):
    triangle_img = draw_random_triangle()
    cv2.imwrite(f'{triangle_dir}/triangle_{i}.png', triangle_img)
    
    rectangle_img = draw_random_rectangle()
    cv2.imwrite(f'{rectangle_dir}/rectangle_{i}.png', rectangle_img)
    
    circle_img = draw_random_circle()
    cv2.imwrite(f'{circle_dir}/circle_{i}.png', circle_img)

print("Images have been successfully saved.")

Images have been successfully saved.


# 데이터셋 클래스 정의

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ShapeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['triangle_images', 'rectangle_images', 'circle_images']
        self.image_paths = []
        self.labels = []

        for idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                if img_name.endswith('.png'):
                    self.image_paths.append(os.path.join(class_dir, img_name))
                    self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


# 데이터 로더 준비

In [22]:
from sklearn.model_selection import train_test_split

# 데이터셋 인스턴스 생성
dataset = ShapeDataset(root_dir='/home/juneyub12/Desktop/sample', transform=transform)

# 데이터셋 인덱스 분리
indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=0.2, stratify=dataset.labels)

# 학습용 데이터로더
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler, num_workers=2)

# 검증용 데이터로더
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler, num_workers=2)


# 모델 정의

In [23]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()


# 학습

In [24]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

learning_rate = 0.001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 검증 루프
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

print('Finished Training')



Epoch [1/10], Loss: 0.8046
Validation Loss: 0.4687, Validation Accuracy: 81.00%
Epoch [2/10], Loss: 0.3189
Validation Loss: 0.3705, Validation Accuracy: 81.83%
Epoch [3/10], Loss: 0.1316
Validation Loss: 0.0851, Validation Accuracy: 97.00%
Epoch [4/10], Loss: 0.0601
Validation Loss: 0.0733, Validation Accuracy: 97.33%
Epoch [5/10], Loss: 0.0259
Validation Loss: 0.0773, Validation Accuracy: 97.00%
Epoch [6/10], Loss: 0.0148
Validation Loss: 0.0451, Validation Accuracy: 97.83%
Epoch [7/10], Loss: 0.0067
Validation Loss: 0.0657, Validation Accuracy: 97.33%
Epoch [8/10], Loss: 0.0036
Validation Loss: 0.0698, Validation Accuracy: 97.33%
Epoch [9/10], Loss: 0.0024
Validation Loss: 0.0441, Validation Accuracy: 98.17%
Epoch [10/10], Loss: 0.0092
Validation Loss: 0.0644, Validation Accuracy: 97.33%
Finished Training


# 평가

In [25]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the {total} validation images: {100 * correct / total:.2f}%')


Accuracy of the model on the 600 validation images: 97.33%
